In [308]:
sums = []
with open('input.txt') as f:
    for line in f:
        sums.append(line.strip())

In [286]:
import re

In [287]:
token = re.compile(r'([0-9]+|[+]|[*]|\(|\))')

In [288]:
debug = False

In [289]:
def evaluate(tokens, depth=0):
    indent = '  '*depth
    if debug: print(indent, tokens)
    start = True
    result = None
    operator = None
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if debug: print(indent, '-', token, start, result, operator)
        if token.isdigit():
            number = int(token)
            if start:
                result = number
                start = False
            else:
                if operator == '+':
                    result += number
                elif operator == '*':
                    result *= number
        elif token in '*+':
            operator = token
        elif token == '(':
            parens = 1
            j = i+1
            while parens > 0:
                subtoken = tokens[j]
                if subtoken == '(':
                    parens += 1
                if subtoken == ')':
                    parens -= 1
                j += 1
            number = evaluate(tokens[i+1:j-1], depth+1)
            i = j-1
            if start:
                result = number
                start = False
            else:
                if operator == '+':
                    result += number
                elif operator == '*':
                    result *= number   
        i += 1
    if debug: print(indent, '= ', result)
    return result

In [290]:
def tokenize(s):
    return token.findall(s)

In [291]:
evaluate(tokenize(s))

13632

In [292]:
expecteds = (26, 437, 12240, 13632)


In [293]:
# run with test data
# for s, e in zip(sums, expecteds):
#     print(s)
#     print(evaluate(tokenize(s)), e)

In [294]:
def scanforclosing(tokens, location, open, closed, direction):
    parens = 1
    while parens > 0:
        location += direction

        if tokens[location] == open:
            parens += 1
        elif tokens[location] == closed:
            parens -= 1
            
    return location

In [295]:
tokens = tokenize('1 * 1 + 1 + 1')

In [296]:
def addparens(tokens, location):
    tokens = tokens.copy()
#     print(tokens[location])
    # add left paren
    if tokens[location - 1].isdigit():
        left = location - 1
    elif tokens[location - 1] == ')':
        left = scanforclosing(tokens, location - 1, ')', '(', -1)
        
    # add right paren
    if tokens[location + 1].isdigit():
        right = location + 1
    else:
        right = scanforclosing(tokens, location + 1, '(', ')', 1)

    tokens.insert(left, '(')
    tokens.insert(right+2, ')')
    
    return tokens

In [297]:
def fixplus(tokens):
    i = 0
    while i < len(tokens):
        if tokens[i] == '+':
            tokens = addparens(tokens, i)
            i += 1
        
        i += 1

    return tokens

In [298]:
tokens

['1', '*', '1', '+', '1', '+', '1']

In [299]:
fixplus(tokens)

['1', '*', '(', '(', '1', '+', '1', ')', '+', '1', ')']

In [300]:
s

'((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'

In [301]:
expecteds = (51, 46, 1445, 669060, 23340)


In [302]:
s = '1 + 2 * 3 + 4 * 5 + 6'

In [303]:
' '.join(fixplus(tokenize(s)))

'( 1 + 2 ) * ( 3 + 4 ) * ( 5 + 6 )'

In [304]:
evaluate(fixplus(tokenize(s)))

231

In [307]:
# run with test data
for s, e in zip(sums, expecteds):
    print(s)
    tokens = fixplus(tokenize(s))
    print('=>', ' '.join(tokens))
    print(evaluate(tokens), e)

9 + 3 * (9 * (9 + 3 + 5 * 3) * 5 * 3 * 5)
=> ( 9 + 3 ) * ( 9 * ( ( ( 9 + 3 ) + 5 ) * 3 ) * 5 * 3 * 5 )
413100 51
5 + 9 * 7 + 5 + 7 + (6 * (4 * 9 + 5 * 7 + 7 + 7) * (7 * 7 + 9 * 5) * 6 + 4 * 9)
=> ( 5 + 9 ) * ( ( ( 7 + 5 ) + 7 ) + ( 6 * ( 4 * ( 9 + 5 ) * ( ( 7 + 7 ) + 7 ) ) * ( 7 * ( 7 + 9 ) * 5 ) * ( 6 + 4 ) * 9 ) )
4978713866 46
3 + (5 * 3 + 5 + 5 * 2 + 2) * 8 * (7 * 7 * 7 * 2)
=> ( 3 + ( 5 * ( ( 3 + 5 ) + 5 ) * ( 2 + 2 ) ) ) * 8 * ( 7 * 7 * 7 * 2 )
1443344 1445
(7 * (4 + 8 + 6)) + 5 + (5 * 6 + 6 + (6 * 7 + 7 + 3 * 6 + 3) * 5 * (5 + 6 * 8)) + 5 + 4
=> ( ( ( ( ( 7 * ( ( ( 4 + 8 ) + 6 ) ) ) + 5 ) + ( 5 * ( ( 6 + 6 ) + ( 6 * ( ( 7 + 7 ) + 3 ) * ( 6 + 3 ) ) ) * 5 * ( ( 5 + 6 ) * 8 ) ) ) + 5 ) + 4 )
2046140 669060
(5 + 2 * (3 + 5 * 3 + 7 + 6 * 8) * 9 + 6) + (7 * 9 + 3 * 2 + 4) * 2 * 6
=> ( ( ( 5 + 2 ) * ( ( 3 + 5 ) * ( ( 3 + 7 ) + 6 ) * 8 ) * ( 9 + 6 ) ) + ( 7 * ( 9 + 3 ) * ( 2 + 4 ) ) ) * 2 * 6
1296288 23340


In [309]:
sum(evaluate(fixplus(tokenize(s))) for s in sums)

323802071857594